<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Teste_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rotinas de Criação de Matriz de Embeddings


In [ ]:
! pip install keras==2.7.0
! pip show keras

     |████████████████████████████████| 1.3 MB 27.4 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 requires keras<2.9,>=2.8.0rc0, but you have keras 2.7.0 which is incompatible.


Name: keras
Version: 2.7.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: tensorflow, keras-vis


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy
import itertools
import nltk
import string, re
import pandas as pd
from numpy import array
from numpy import asarray
from numpy import loadtxt
from numpy import savetxt
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text, sequence

In [3]:
########## PATHS DE ARQUIVOS ##########
# 
PATH_CBOW_s300 = r"/content/drive/MyDrive/TCC/dados/word_embeddings/cbow_s300_modifyed.txt"

#
PATH_BASE_1_LEILA_CLASSIFICADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1JuntaHateENotHateDaLeila_balanceada.csv" #entrada
PATH_MATRIZ_CBOW_300_BASE_1 = r"/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base1JuntaHateENotHateDaLeila_balanceada_CBOW300.CSV" #saida
#
PATH_BASE_2_LEILA_CLASSIFICADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_maior4_menor25_limpissima_balanceada.csv" #entrada
PATH_MATRIZ_CBOW_300_BASE_2_LEILA = r"/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_maior4_menor25_limpissima_balanceada_CBOW300.CSV" #saida

PATH_BASE_2_CLASSIFICADA_BALANCEADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_classificada_balanceada.csv" #entrada
PATH_MATRIZ_CBOW_300_BASE_2 = r"/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_classificada_balanceada_CBOW300.csv" #saida

In [4]:
#DataFrame_Base= pd.read_csv(PATH_BASE_1_CLASSIFICADA)
DataFrame_Base = pd.read_csv(PATH_BASE_2_CLASSIFICADA_BALANCEADA)
DataFrame_Base.head()

,Unnamed: 0,text,label
0,0,retwet bahia fazer sendo governada pt sei baia...,1
1,1,k imagine atitude mental negativa f idiota fec...,1
2,2,general heleno é bolsonaro rosna late late lat...,1
3,3,alan ser cara pau achar havendo ruptura nesse ...,1
4,4,canalhascomunistas caçarão chapa presidencialn...,1


In [5]:
palavras_unicas = list(DataFrame_Base['text'].str.split(' ', expand=True).stack().unique())
print(len(palavras_unicas))
# retira eventuais numeros. Numeros unicos por algum motivo matam o processo de filtragem. 
palavras_unicas = set(map(lambda x: re.findall(r'^\D+', x)[0] if re.findall(r'^\D+', x) != [] else "a" ,palavras_unicas))
print(len(palavras_unicas))

19043
19043


# Filtra os embeddings

In [6]:
# Código para filtrar o arquivo de embeddings.

embeddings_index = dict()

f = open(PATH_CBOW_s300, encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    if (word in palavras_unicas):
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
f.close()

In [7]:
def monta_linha(palavra, coeficientes):
    # Monta a linha com palavra _ coeficientes.
    
    linha = f'{palavra}'
    for coef in coeficientes:
        linha += f' {coef:.5f}'
    linha += '\n'
    
    return linha

In [8]:
lista_de_embeddings = []
for word in embeddings_index:
    coeficientes = embeddings_index[word]
    linha = monta_linha(word, coeficientes)
    lista_de_embeddings.append(linha)

In [9]:
embeddings_dict = {}
for i, line in enumerate(lista_de_embeddings):
    values = line.split()
    embeddings_dict[values[0]] = numpy.asarray(values[1:], dtype='float32')

In [10]:
token = text.Tokenizer()
token.fit_on_texts(palavras_unicas)
#word_index: dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.
word_index = token.word_index

In [11]:
# create token-embedding mapping

lista_de_chaves = list(embeddings_dict.keys()) #palavras

tamanho_da_dimensao = len(embeddings_dict[lista_de_chaves[0]])

embedding_matrix = numpy.zeros((len(word_index) + 1, tamanho_da_dimensao ))

for word, i in word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [12]:
print(word_index)

{'esculhambam': 1, 'antagonista': 2, 'dê': 3, 'remember': 4, 'abrer': 5, 'convém': 6, 'retrata': 7, 'piranha': 8, 'hahahahahah': 9, 'pratique': 10, 'látex': 11, 'polemica': 12, 'notar': 13, 'pareceu': 14, 'roubalheira': 15, 'fascistinha': 16, 'harmonizaçãovinhos': 17, 'sumam': 18, 'insurgência': 19, 'textos': 20, 'passarem': 21, 'juntou': 22, 'adjetivos': 23, 'parcelamos': 24, 'desprotegido': 25, 'fodida': 26, 'uau': 27, 'péssima': 28, 'joguem': 29, 'personalizado': 30, 'eliéser': 31, 'carne': 32, 'marina': 33, 'notificação': 34, 'vassoura': 35, 'esmurrar': 36, 'ir': 37, 'fantástico': 38, 'isaías': 39, 'repararam': 40, 'submisso': 41, 'barriga': 42, 'gatorade': 43, 'inflacionário': 44, 'bonda': 45, 'gerando': 46, 'carrão': 47, 'dramas': 48, 'tímida': 49, 'causa': 50, 'demonstrado': 51, 'desinformada': 52, 'sydney': 53, 'segredo': 54, 'completa': 55, 'parafraseando': 56, 'câmera': 57, 'reúne': 58, 'fiec': 59, 'vedadeiros': 60, 'respeitosamente': 61, 'skipjump': 62, 'ko': 63, 'consequênc

In [13]:
print(type(embedding_matrix))
embedding_matrix.shape

(19044, 300)

<class 'numpy.ndarray'>


In [14]:
print(embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.37270999  0.02985     0.04875    ...  0.03141    -0.49873999
   0.41057   ]
 ...
 [-0.20081     0.10339     0.46324    ...  0.04749    -0.07389
   0.31347999]
 [-0.22578     0.00257     0.14324    ... -0.13773     0.2137
  -0.33302999]
 [ 0.31101     0.07585    -0.12949    ... -0.06088    -0.27162999
   0.02321   ]]


In [15]:
# save to csv file
savetxt(PATH_MATRIZ_CBOW_300_BASE_2, embedding_matrix, delimiter=',')

In [16]:
# data = loadtxt('data.csv', delimiter=',')
# # print the array
# print(data)